In [54]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim  as optim
from PIL import Image

In [2]:
from torchvision import datasets,transforms
from torch.utils.data import DataLoader


In [3]:

transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
])


training_datasets = datasets.CIFAR10(root="data",train=True,download=True,transform=transformer)
test_datasets = datasets.CIFAR10(root="data",train=True,download=True,transform=transformer)

In [4]:
training_loader = DataLoader(training_datasets,shuffle=True,batch_size=32)
test_loader = DataLoader(test_datasets,shuffle=True,batch_size=32)

In [5]:
training_loader

In [6]:
features,label = training_datasets[0]
features.shape

torch.Size([3, 32, 32])

In [79]:
class_names = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]										

In [8]:
class MyNeuralNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,12,5) #Creates a new layer
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(12,24,5)
        self.fc1 = nn.Linear(24*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,features):
        features = self.pool(F.relu(self.conv1(features)))
        features = self.pool(F.relu(self.conv2(features)))
        features = torch.flatten(features,1)
        features = F.relu(self.fc1(features))
        features = F.relu(self.fc2(features))
        features = self.fc3(features)
        return features        
        


In [9]:
cnn_model = MyNeuralNetwork()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_model.parameters(),lr=0.001,momentum=0.9)

In [10]:
for epoch in range(30):
    print(f"=== TRAINING EPOCHS ... {epoch} ===")

    running_loss = 0.0

    for i,data in enumerate(training_loader):
        inputs,labels = data
        optimizer.zero_grad()
        outputs = cnn_model(inputs)
        loss = loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss =+ loss.item()
        
    print(f"Loss : {running_loss/len(training_loader)}")
    

=== TRAINING EPOCHS ... 0 ===
Loss : 0.0012251514695961355
=== TRAINING EPOCHS ... 1 ===
Loss : 0.0008833401491454376
=== TRAINING EPOCHS ... 2 ===
Loss : 0.0007277329000081302
=== TRAINING EPOCHS ... 3 ===
Loss : 0.0008259520497142087
=== TRAINING EPOCHS ... 4 ===
Loss : 0.0012597491629826893
=== TRAINING EPOCHS ... 5 ===
Loss : 0.000615860091823839
=== TRAINING EPOCHS ... 6 ===
Loss : 0.0007993946911353601
=== TRAINING EPOCHS ... 7 ===
Loss : 0.0007135649178932664
=== TRAINING EPOCHS ... 8 ===
Loss : 0.00043288340419054183
=== TRAINING EPOCHS ... 9 ===
Loss : 0.0004912078266180409
=== TRAINING EPOCHS ... 10 ===
Loss : 0.0006737899719257806
=== TRAINING EPOCHS ... 11 ===
Loss : 0.0005184953516290802
=== TRAINING EPOCHS ... 12 ===
Loss : 0.0005247126537794039
=== TRAINING EPOCHS ... 13 ===
Loss : 0.0005364897383838148
=== TRAINING EPOCHS ... 14 ===
Loss : 0.0005307355830094331
=== TRAINING EPOCHS ... 15 ===
Loss : 0.0005099782940674805
=== TRAINING EPOCHS ... 16 ===
Loss : 0.0002942762

In [16]:
training_loader

In [20]:
torch.save(cnn_model.state_dict(),"trained_neural.pth")

In [24]:
cnn_model = MyNeuralNetwork()
cnn_model.load_state_dict(torch.load("trained_neural.pth"))

<All keys matched successfully>

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
correct = 0
total = 0

cnn_model.eval()

with torch.no_grad():
    for datas,labels in test_loader:
        datas,labels = datas.to(device), labels.to(device)
        outputs = cnn_model(datas)
        _, predicted = torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
    accuracy = 100* correct/total

    print(f"Accuracy of the model : {accuracy:.2f}%")
    
    

Accuracy of the model : 84.76%


In [107]:
new_transformer = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5]),
])


def Image_Loader (image_path): 
    image = Image.open(image_path)
    image = new_transformer(image)
    image = image.unsqueeze(0)
    return image

my_input = Image_Loader("Golden+Retrievers+dans+pet+care.jpeg")
airplane_image = Image_Loader("images.jpg")
dog_image = Image_Loader("dog_2.jpg")


#prediction over here
with torch.no_grad():
    output = cnn_model(airplane_image)
    _, predicted = torch.max(output,1)
    print(f"Prediction : {class_names[predicted.item()]}")
    print(f"Item : {predicted.item()}")
    print(f"Predicted Tensors of the Images : {predicted}")
    

Prediction : airplane
Item : 0
Predicted Tensors of the Images : tensor([0])


In [109]:
output = cnn_model(dog_image)
_, predicted = torch.max(output,1)
print(f"Prediction : {class_names[predicted.item()]}")

Prediction : dog


In [101]:
dog_image

tensor([[[[-0.4118, -0.3490, -0.2784,  ..., -0.2078, -0.1843, -0.1529],
          [-0.4510, -0.4039, -0.3569,  ..., -0.2314, -0.2000, -0.1765],
          [-0.4980, -0.4667, -0.4431,  ..., -0.2627, -0.2314, -0.2157],
          ...,
          [-0.1608, -0.2000, -0.2235,  ..., -0.1608, -0.2314, -0.3255],
          [-0.1529, -0.1765, -0.2235,  ..., -0.4431, -0.4745, -0.5059],
          [-0.6863, -0.6941, -0.7333,  ..., -0.2235, -0.4196, -0.3961]],

         [[-0.6078, -0.5765, -0.5529,  ..., -0.4745, -0.4588, -0.4196],
          [-0.6392, -0.6157, -0.6000,  ..., -0.4902, -0.4745, -0.4353],
          [-0.6706, -0.6627, -0.6471,  ..., -0.5059, -0.4824, -0.4588],
          ...,
          [-0.3333, -0.3725, -0.3882,  ..., -0.3725, -0.4196, -0.5059],
          [-0.3412, -0.3569, -0.3882,  ..., -0.5843, -0.6157, -0.6471],
          [-0.7490, -0.7725, -0.8039,  ..., -0.4039, -0.5451, -0.5294]],

         [[-0.7961, -0.7725, -0.7490,  ..., -0.6314, -0.6157, -0.5765],
          [-0.8275, -0.8039, -